## Playground

### Todo
- [x] eine Funktion die ein Lat/Long tuple annimmt und die Urls der n nächsten Radiostationen zurückgibt.

- [ ] eine Funktion die die Koordinaten aus dem erhaltenen Format zu Lat/Long übersetzt.

- [x] eine Funktion die die ID einer gewählten Radiostation annimmt und den Audiostream dieser Station ausgibt.

- [ ] eine Funktion die den Videostream einer Webcam annimmt und Koordinaten ausliest (auf welche Art auch immer).

In [3]:
import json
import requests
import vlc

'''
getting basic list of places. geo field ist a tuple of shape [Long, Lat]
'''

r = requests.get('http://radio.garden/api/ara/content/places', headers={'Accept': 'application/json'})
r_json = r.json()
place_list = r_json["data"]["list"]
n_stations = 20

instance = vlc.Instance('--input-repeat=-1', '--fullscreen')
player=instance.media_player_new()

In [ ]:
import geopy.distance

'''
distances of places to given lat/long tuple
'''

pin_location = (48.2083537,16.3725042) #Vienna
distance_list = [dict(place, distance=geopy.distance.geodesic((place["geo"][1],place["geo"][0]), pin_location).km)  for place in place_list]

In [ ]:
'''
Sorting by closest places to given lat/long tuple & selecting the until n amount of radio stations is reached
'''

distance_list = sorted(distance_list, key=lambda x: x['distance'])
selected_places = []
for place in distance_list:
    if sum(selected_place['size'] for selected_place in selected_places) < n_stations:
        selected_places.append(place)

In [ ]:
import random
from datetime import datetime

'''
selecting n radio stations from the selected places
'''

random.seed(str(datetime.now())) #random seed generated by system time
selected_channels = []
for place in selected_places:
    #radio stations are not saved within the place data therefore a new request must be sent per selected place
    r = requests.get(f'http://radio.garden/api/ara/content/page/{place["id"]}/channels', headers={'Accept': 'application/json'}) 
    r_json = r.json()

    #if the furthest of the selected places contains more stations than requested by n, we randomly select until n is met
    if place == selected_places[len(selected_places)-1] and place["size"] > 1:
        for channel in random.sample(r_json["data"]["content"][0]["items"], n_stations-len(selected_channels)):
            selected_channels.append(channel)

    else:
        for channel in r_json["data"]["content"][0]["items"]:
            selected_channels.append(channel)

In [ ]:
import vlc
import random
'''
getting audio output from Station ID
'''

random_channel = random.choice(selected_channels)
print(list(random_channel.values())[1])
some_ID = list(random_channel.values())[0][-8::1]  #OE1
  
url =f"http://radio.garden/api/ara/content/listen/{some_ID}/channel.mp3"


#instance = vlc.Instance('--input-repeat=-1', '--fullscreen')

#Define VLC player
#player=instance.media_player_new()

#Define VLC media
media=instance.media_new(url)

#Set player media
player.set_media(media)

#Play the media
player.play()

#Pause the media

#player.pause()




In [ ]:
#Stop the media
player.stop()

In [1]:
%matplotlib tk
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pynput.mouse import Controller, Listener

mpl.rcParams['toolbar'] = 'None'

fig = plt.figure()

img=mpimg.imread('plattkarte_stretched.jpg')

imgplot = plt.imshow(img)
imgplot.axes.get_xaxis().set_visible(False)
imgplot.axes.get_yaxis().set_visible(False)

mng = plt.get_current_fig_manager()
### works on Ubuntu??? >> did NOT working on windows
# mng.resize(*mng.window.maxsize())
 #works fine on Windows!
mng.full_screen_toggle()
#fig.tight_layout(pad=0) #this is wonky af. 
plt.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0) #this has to be set individually to screen



plt.show()

In [4]:
import geopy.distance
import random
from datetime import datetime
import vlc

def get_station(lat, long):
    player.stop()
    random.seed(str(datetime.now()))
    distance_list = [dict(place, distance=geopy.distance.geodesic((place["geo"][1],place["geo"][0]), (long,lat)).km)  for place in place_list]
    distance_list = sorted(distance_list, key=lambda x: x['distance'])

    selected_places = []
    for place in distance_list:
        if sum(selected_place['size'] for selected_place in selected_places) < n_stations:
            selected_places.append(place)
    selected_channels = []
    for place in selected_places:
        #radio stations are not saved within the place data therefore a new request must be sent per selected place
        r = requests.get(f'http://radio.garden/api/ara/content/page/{place["id"]}/channels', headers={'Accept': 'application/json'}) 
        r_json = r.json()

        #if the furthest of the selected places contains more stations than requested by n, we randomly select until n is met
        if place == selected_places[len(selected_places)-1] and place["size"] > 1:
            for channel in random.sample(r_json["data"]["content"][0]["items"], n_stations-len(selected_channels)):
                selected_channels.append(channel)

        else:
            for channel in r_json["data"]["content"][0]["items"]:
                selected_channels.append(channel)
    random_channel = random.choice(selected_channels)
    print(list(random_channel.values())[1])
    some_ID = list(random_channel.values())[0][-8::1]  #OE1
    
    url =f"http://radio.garden/api/ara/content/listen/{some_ID}/channel.mp3"


    media=instance.media_new(url)
    player.set_media(media)
    player.play()



In [6]:
from pynput.mouse import Controller, Listener

def on_click(x, y, button, pressed):
    if pressed:
        print ('Mouse clicked at ({0}, {1}) with {2}'.format(x, y, button))
        get_station((x-1280)/7.11111111,((y*-1)+720)/8)


with Listener(on_click=on_click) as Listener:
    Listener.join()

Mouse clicked at (1380, 250) with Button.left
Vättervåg FM 98,5


Unhandled exception in listener callback
Traceback (most recent call last):
  File "c:\Repos\RadioWall\.venv\lib\site-packages\pynput\_util\win32.py", line 386, in _handler
    converted = self._convert(code, msg, lpdata)
  File "c:\Repos\RadioWall\.venv\lib\site-packages\pynput\_util\win32.py", line 401, in _convert
    raise NotImplementedError()
NotImplementedError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Repos\RadioWall\.venv\lib\site-packages\pynput\_util\__init__.py", line 228, in inner
    return f(self, *args, **kwargs)
  File "c:\Repos\RadioWall\.venv\lib\site-packages\pynput\_util\win32.py", line 390, in _handler
    self._handle(code, msg, lpdata)
  File "c:\Repos\RadioWall\.venv\lib\site-packages\pynput\mouse\_win32.py", line 212, in _handle
    self.on_click(data.pt.x, data.pt.y, button, pressed)
  File "c:\Repos\RadioWall\.venv\lib\site-packages\pynput\_util\__init__.py", line 144, in inner
    if 